#### Spotify API:
- API connection
- Data sorting
- Data to CSV

In [1]:
# API connection

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


CLIENT_ID = '3f8cc110d1534d4ba1cd6d063cf114e3'
CLIENT_SECRET = '2c63ffda0e7d4247aaa719de974460bf'


auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [2]:
# Data sorting

spotify_data = []

def data_extraction_s(list_url):

    for url in list_url:

        playlist = sp.playlist(url, fields=None, market=None, additional_types= 'track')

        
        for item in playlist["tracks"]["items"]:
            track = item["track"]
            artist = item["track"]["artists"]
            eurovisionyear = playlist['name']

            spotify_data.append({
                "artists_name": artist[0]["name"].title(),
                "track_name": track["name"].title(),
                "popularity": track["popularity"],
                "duration": track["duration_ms"],
                "eurovision_year" : eurovisionyear.title()
                })
        
        

In [3]:
# Data to CSV

raw_data_s = data_extraction_s([
    "https://open.spotify.com/playlist/3ZdQUt8Tmtt7oOU8UM2koe?si=eea27348cf654d21", 
    "https://open.spotify.com/playlist/5h0sQpJnLVzgy5iOH1UNcl?si=3a1d926528484506", 
    "https://open.spotify.com/playlist/3EBtzzGyrlDUr1QvXVGz0h?si=DMNkLRIjQE6THtHUYc-oQw",
    "https://open.spotify.com/playlist/61fyG82H3Cha91cgKgQT6S?si=o-Xs3DfHRZyRuR8qaY9mxAW",
    "https://open.spotify.com/playlist/2HMW5JNZNh9HLfIHNzY0DP?si=f29ed55294394682"
    ])

df_spotify = pd.DataFrame(spotify_data)
df_spotify

df_spotify.to_csv('Spotify_data.csv')


In [4]:
df_spotify

,artists_name,track_name,popularity,duration,eurovision_year
0,Chingiz,Truth - Radio Edit,0,209411,Eurovision 2019
1,John Lundvik,Too Late For Love,0,178133,Eurovision 2019
2,Kate Miller-Heidke,Zero Gravity,38,177443,Eurovision 2019
3,Michela,Chameleon,0,179698,Eurovision 2019
4,Mahmood,Soldi,0,195476,Eurovision 2019
...,...,...,...,...,...
205,Fahree,Özünlə Apar,40,182090,Eurovision 2024
206,Sarah Bonnici,Loop,42,179727,Eurovision 2024
207,Megara,11:11,39,180087,Eurovision 2024
208,Natalia Barbu,In The Middle,37,178853,Eurovision 2024


#### Eurovision API:
- Api connection & Sorting
- Data to CSV

In [10]:
import requests
import pandas as pd
import json

In [11]:
# API connection & Sorting

eurovision_data = []
def data_extraction_e(list_years):

    eu_data = requests.get(f"https://eurovisionapi.runasp.net/api/contests/{list_years}/")

    eu_data_json = eu_data.json()
    
    if 'contestants' in eu_data_json:
        
        for idx, i in enumerate(eu_data_json["contestants"], start=1):

            contestant_name = i["artist"]
            song_name = i["song"]
            country = i["country"]

            eurovision_data.append({
                "Ranking": idx,
                "Artist": contestant_name.title(),
                "Track": song_name.title(),
                "Country": country.upper(),
                "Eurovision_Year": list_years
                })

In [31]:
# Data to CSV

eurovision_years= [2019,2021,2022,2023,2024]

for year in eurovision_years:
    raw_data_e = data_extraction_e(year)
df_eurovision = pd.DataFrame(eurovision_data)
df_eurovision

df_eurovision.to_csv('Eurovision_data.csv', index=False)


#### MySQL Data Base
- MySQL connection
- Schema Creation and Use
- Tables Creation

In [19]:
# MySQL connection

import mysql.connector

try:
    cnx = mysql.connector.connect(user='root', password='#23Janeiro23#',
                              host='127.0.0.1')
    

    mycursor = cnx.cursor()
    

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

In [30]:
# Schema Creation and Use

query_create_schema = """CREATE SCHEMA StreamMusic"""
mycursor.execute(query_create_schema)
mycursor.execute('USE StreamMusic')

In [ ]:
# Tables creation


the_tables = {  'API_Eurovision' :          """CREATE TABLE api_eurovision (
                                                EurovisionYear YEAR NOT NULL,
                                                Ranking INT NOT NULL,
                                                Artist VARCHAR(45) NOT NULL,                              
                                                Track VARCHAR(70) NOT NULL,
                                                Country VARCHAR(3) NOT NULL)""",

                'API_Spotify' :              """CREATE TABLE api_spotify_playlist (
                                                Artist VARCHAR(45),
                                                Track VARCHAR(70),
                                                Popularity INT NOT NULL,
                                                Duration INT NOT NULL,
                                                EurovisionYear VARCHAR(25) NOT NULL,
                                                PRIMARY KEY (`Artist`, `Track`)
                                                )""", 
                
                'API_LastFM_Info_Artists' :  """CREATE TABLE api_lastfm_info_artists (
                                                Artist VARCHAR(45),
                                                Listeners INT,
                                                Playcount INT,
                                                Genre1 VARCHAR(45),
                                                Genre2 VARCHAR(45),
                                                Genre3 VARCHAR(45),
                                                Similar_Artists1 VARCHAR(45),
                                                Similar_Artists2 VARCHAR(45),
                                                Similar_Artists3 VARCHAR(45),
                                                PRIMARY KEY (`Artist`),
                                                CONSTRAINT fk_artists_from_s
                                                        FOREIGN KEY Artist
                                                        REFERENCES api_spotify_playlist (Artist)
                                                        ON UPDATE CASCADE
                                                        ON DELETE CASCADE
                                                )""",
                
                'API_LastFM_Top_Tracks' :    """CREATE TABLE api_lastfm_top_tracks (
                                                Release_year YEAR NOT NULL,
                                                Artist VARCHAR(70) NOT NULL,
                                                Top_track VARCHAR(150),
                                                PRIMARY KEY (`Artist`),
                                                CONSTRAINT fk_artists_from_s
                                                        FOREIGN KEY Artist
                                                        REFERENCES api_spotify_playlist (Artist)
                                                        ON UPDATE CASCADE
                                                        ON DELETE CASCADE
                                                )""" 
                }



for k, v in the_tables.items():
        print(f'Table created: {k}')
        mycursor.execute(v)

        print('------------')



Table created: API_Eurovision
------------
Table created: API_Spotify
------------
Table created: API_LastFM_Info_Artists


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'REFERENCES api_spotify_playlist (Artist)
                                       ' at line 13

#### Data Import

- Method
- Queries
- Data Import

In [16]:
# Data Import Method

def data_import(query,values):

    mycursor = cnx.cursor()
    mycursor.executemany(query, values)
    print(mycursor.rowcount,'values entered')
    cnx.commit()
 

In [ ]:
# Data Import Queries 

query_e_import =            """ INSERT INTO API_EUROVISION (EurovisionPosition, Artist, Track, Country, Eurovision_Year)
                                VALUES (%s, %s, %s, %s, %s, %s) """

query_s_import =            """ INSERT INTO api_spotify_playlist (Artist, Track, Popularity, Duration, EurovisionYear)
                                VALUES (%s, %s, %s, %s, %s) """

query_l_artist_import =     """ INSERT INTO api_lastfm_info_artists (Artist, Listeners, Playcount, Genre1, Genre2, Genre3, Similar_Artists1, Similar_Artists2, Similar_Artists3)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) """

query_l_toptracks_import =  """ INSERT INTO api_lastfm_top_tracks (Release_year, Artist, Top_track)
                                VALUES (%s, %s, %s) """



In [32]:
# Eurovision Data Import

e_file = pd.read_csv('Eurovision_data.csv',index_col=0)

e_values = list(e_file.itertuples(index=False, name=None))
eurovisionimp = data_import(query_e_import, e_values)

InterfaceError: Failed executing the operation; Not enough parameters for the SQL statement

In [ ]:
# Spotify Data Import

s_file = pd.read_csv('Spotify_data.csv',index_col=0)

s_values = list(s_file.itertuples(index = False, name = None))
spotifyimp = data_import(query_s_import, s_values)

210 values entered
